In [ ]:
import sysimport subprocesspackages = [    "yolov5",    "opencv-python",    "tqdm",    "numpy",    "Pillow"]for package in packages:    try:        __import__(package.replace("-", "_"))        print(f"{package} already installed")    except ImportError:        print(f"Installing {package}...")        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

In [ ]:
import sysimport subprocesstry:    import insightface    print("InsightFace already installed")except ImportError:    print("Installing InsightFace...")    try:        subprocess.check_call([sys.executable, "-m", "pip", "install", "onnxruntime"])        subprocess.check_call([sys.executable, "-m", "pip", "install",                              "insightface", "--no-build-isolation", "--no-cache-dir"])        import insightface        print("InsightFace installed successfully")    except Exception as e:        print(f"Could not install InsightFace automatically: {e}")        print("Please install manually:")        print("  pip install onnxruntime")        print("  pip install insightface --no-build-isolation")        print("\nNote: If you encounter DLL errors, install Visual C++ Redistributable 2019")        print("Download from: https://aka.ms/vs/17/release/vc_redist.x64.exe")

In [ ]:
import osWEIGHTS_PATH = r"d:\AI_Diploma\AI_diploma\Computer_Vision\Projects\insightface\Face_detection\weights\best.pt"DATASET_IMAGES_DIR = r"d:\AI_Diploma\AI_diploma\Computer_Vision\Projects\insightface\data"DETECT_RUN_DIR = r"d:\AI_Diploma\AI_diploma\Computer_Vision\Projects\insightface\runs_detect\faces"DATA_ROOT = r"d:\AI_Diploma\AI_diploma\Computer_Vision\Projects\insightface\Face_detection"FACES_DIR = os.path.join(DATA_ROOT, "faces_cropped")LABELS_DIR = os.path.join(DETECT_RUN_DIR, "labels")EMB_DB_PATH = os.path.join(DATA_ROOT, "embeddings_raw.pkl")os.makedirs(os.path.dirname(DETECT_RUN_DIR), exist_ok=True)os.makedirs(DATA_ROOT, exist_ok=True)print("Paths configured successfully")print(f"Weights: {WEIGHTS_PATH}")print(f"Dataset: {DATASET_IMAGES_DIR}")print(f"Faces: {FACES_DIR}")print(f"Labels: {LABELS_DIR}")print(f"Embeddings DB: {EMB_DB_PATH}")

In [ ]:
import yolov5import torchimport osif not os.path.exists(WEIGHTS_PATH):    print("ERROR: Weights file not found!")    print(f"Expected path: {WEIGHTS_PATH}")else:    print("Loading YOLOv5 model...")    print(f"File size: {os.path.getsize(WEIGHTS_PATH) / (1024*1024):.2f} MB")        try:        model = yolov5.load(WEIGHTS_PATH)        print("Model loaded successfully")        print(f"Model type: {type(model)}")    except Exception as e:        print(f"Error loading model: {e}")

In [ ]:
import globimport shutilimport osprint("=" * 60)print("Deleting old labels and cropped faces")print("=" * 60)if os.path.exists(LABELS_DIR):    label_files = glob.glob(os.path.join(LABELS_DIR, "*.txt"))    print(f"Found {len(label_files)} label files")    for label_file in label_files:        try:            os.remove(label_file)        except Exception as e:            print(f"Error deleting {label_file}: {e}")    print(f"Deleted {len(label_files)} label files")else:    print("No labels directory found")if os.path.exists(FACES_DIR):    face_files = glob.glob(os.path.join(FACES_DIR, "**", "*.jpg"), recursive=True)    print(f"Found {len(face_files)} cropped face files")    try:        shutil.rmtree(FACES_DIR)        print(f"Deleted faces_cropped directory")    except Exception as e:        print(f"Error deleting faces_cropped directory: {e}")else:    print("No faces_cropped directory found")os.makedirs(LABELS_DIR, exist_ok=True)os.makedirs(FACES_DIR, exist_ok=True)print("\nCleanup completed. Ready to process data from scratch.")

In [ ]:
import osimport globimport cv2from tqdm import tqdmimage_patterns = [    os.path.join(DATASET_IMAGES_DIR, "**", "*.jpg"),    os.path.join(DATASET_IMAGES_DIR, "**", "*.jpeg"),    os.path.join(DATASET_IMAGES_DIR, "**", "*.png")]all_images = []for pattern in image_patterns:    all_images.extend(glob.glob(pattern, recursive=True))print(f"Found {len(all_images)} total images")BATCH_SIZE = 16model.conf = 0.25model.iou = 0.45print("\n" + "=" * 60)print("Detecting faces and creating labels")print("=" * 60)successful = 0failed = 0total_detections = 0images_with_faces = 0images_without_faces = 0for i in tqdm(range(0, len(all_images), BATCH_SIZE), desc="Processing batches"):    batch = all_images[i:i+BATCH_SIZE]        for img_path in batch:        try:            result = model(img_path, size=640)                        img = cv2.imread(img_path)            if img is None:                failed += 1                images_without_faces += 1                continue                        h, w = img.shape[:2]                        try:                detections_df = result.pandas().xyxy[0]                if detections_df.empty or len(detections_df) == 0:                    failed += 1                    images_without_faces += 1                    continue            except (AttributeError, IndexError, KeyError):                failed += 1                images_without_faces += 1                continue                        face_detections = detections_df[detections_df['class'] == 0].copy()            if len(face_detections) == 0:                failed += 1                images_without_faces += 1                continue                        images_with_faces += 1                        base_name = os.path.splitext(os.path.basename(img_path))[0]            invalid_chars = ['<', '>', ':', '"', '|', '?', '*']            for char in invalid_chars:                base_name = base_name.replace(char, '_')            base_name = base_name.strip('. ')            if len(base_name) > 200:                base_name = base_name[:200]                        label_file = os.path.join(LABELS_DIR, base_name + ".txt")                        with open(label_file, 'w', encoding='utf-8') as f:                for _, det in face_detections.iterrows():                    x1, y1, x2, y2 = det['xmin'], det['ymin'], det['xmax'], det['ymax']                                        x_center = ((x1 + x2) / 2) / w                    y_center = ((y1 + y2) / 2) / h                    width = (x2 - x1) / w                    height = (y2 - y1) / h                                        f.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")                    total_detections += 1                        successful += 1        except Exception as e:            failed += 1            images_without_faces += 1            continueprint("\n" + "=" * 60)print("Detection Summary")print("=" * 60)print(f"Total images: {len(all_images)}")print(f"Images with faces: {images_with_faces}")print(f"Images without faces: {images_without_faces}")print(f"Successful: {successful}")print(f"Failed: {failed}")print(f"Total face detections: {total_detections}")if os.path.isdir(LABELS_DIR):    label_files = [f for f in os.listdir(LABELS_DIR) if f.endswith('.txt')]    print(f"Generated {len(label_files)} label files")    if len(all_images) > 0:        print(f"Success rate: {len(label_files)}/{len(all_images)} ({100*len(label_files)/len(all_images):.1f}%)")

In [ ]:
import osimport globimport cv2from tqdm import tqdmprint("=" * 60)print("Step 7: Cropping Faces")print("=" * 60)label_files = glob.glob(os.path.join(LABELS_DIR, "*.txt"))print(f"Found {len(label_files)} label files")label_to_image = {}for label_file in label_files:    base_name = os.path.splitext(os.path.basename(label_file))[0]    for img_path in all_images:        img_base = os.path.splitext(os.path.basename(img_path))[0]        invalid_chars = ['<', '>', ':', '"', '|', '?', '*']        for char in invalid_chars:            img_base = img_base.replace(char, '_')            base_name = base_name.replace(char, '_')        if img_base == base_name or img_base.startswith(base_name) or base_name.startswith(img_base):            label_to_image[label_file] = img_path            breakprint(f"Mapped {len(label_to_image)} labels to images")total_faces = 0failed_crops = 0for label_file in tqdm(label_files, desc="Cropping faces"):    if label_file not in label_to_image:        continue        img_path = label_to_image[label_file]        try:        img = cv2.imread(img_path)        if img is None:            failed_crops += 1            continue                h, w = img.shape[:2]                with open(label_file, 'r', encoding='utf-8') as f:            lines = f.readlines()                if len(lines) == 0:            continue                rel_path = os.path.relpath(img_path, DATASET_IMAGES_DIR)        path_parts = [p for p in rel_path.split(os.sep) if p]        person_name = None                for part in reversed(path_parts[:-1]):            if not part.isdigit() and part not in ['Data base', 'Database', 'data']:                person_name = part                break                if person_name is None:            person_name = path_parts[-2] if len(path_parts) >= 2 else "unknown"                invalid_chars = ['<', '>', ':', '"', '|', '?', '*']        for char in invalid_chars:            person_name = person_name.replace(char, '_')        person_name = person_name.strip('. ')        if len(person_name) > 200:            person_name = person_name[:200]                person_dir = os.path.join(FACES_DIR, person_name)        os.makedirs(person_dir, exist_ok=True)                for idx, line in enumerate(lines):            parts = line.strip().split()            if len(parts) != 5:                continue                        x_center = float(parts[1])            y_center = float(parts[2])            width = float(parts[3])            height = float(parts[4])                        x1 = int((x_center - width/2) * w)            y1 = int((y_center - height/2) * h)            x2 = int((x_center + width/2) * w)            y2 = int((y_center + height/2) * h)                        padding = 20            x1 = max(0, x1 - padding)            y1 = max(0, y1 - padding)            x2 = min(w, x2 + padding)            y2 = min(h, y2 + padding)                        face_crop = img[y1:y2, x1:x2]                        if face_crop.size == 0 or face_crop.shape[0] < 30 or face_crop.shape[1] < 30:                continue                        base_name = os.path.splitext(os.path.basename(img_path))[0]            for char in invalid_chars:                base_name = base_name.replace(char, '_')            if len(base_name) > 100:                base_name = base_name[:100]                        face_filename = f"{base_name}_face_{idx:03d}.jpg"            face_path = os.path.join(person_dir, face_filename)                        cv2.imwrite(face_path, face_crop, [cv2.IMWRITE_JPEG_QUALITY, 95])            total_faces += 1        except Exception as e:        failed_crops += 1        if failed_crops <= 5:            print(f"\nError cropping {os.path.basename(img_path)}: {e}")        continueprint("\n" + "=" * 60)print("Cropping complete")print("=" * 60)print(f"Total faces cropped: {total_faces}")print(f"Failed crops: {failed_crops}")print(f"Faces saved to: {FACES_DIR}")if os.path.exists(FACES_DIR):    unique_persons = [d for d in os.listdir(FACES_DIR) if os.path.isdir(os.path.join(FACES_DIR, d))]    print(f"Unique persons: {len(unique_persons)}")

In [ ]:
import sysimport subprocessprint("=" * 60)print("Step 8: Setting up InsightFace")print("=" * 60)try:    import insightface    print("InsightFace already installed")except ImportError:    print("Installing InsightFace...")    try:        subprocess.check_call([sys.executable, "-m", "pip", "install", "onnxruntime"])        subprocess.check_call([sys.executable, "-m", "pip", "install",                             "insightface", "--no-build-isolation", "--no-cache-dir"])        import insightface        print("InsightFace installed successfully")    except Exception as e:        print(f"Error installing InsightFace: {e}")        print("\nPlease try the following:")        print("1. Install Visual C++ Redistributable 2019")        print("   Download: https://aka.ms/vs/17/release/vc_redist.x64.exe")        print("2. Reinstall onnxruntime:")        print("   pip uninstall onnxruntime")        print("   pip install onnxruntime")        raisetry:    app = insightface.app.FaceAnalysis(providers=['CPUExecutionProvider'])    app.prepare(ctx_id=0, det_size=(640, 640))    print("InsightFace model loaded successfully")except Exception as e:    print(f"Error loading InsightFace model: {e}")    print("\nTroubleshooting:")    print("1. Make sure Visual C++ Redistributable 2019 is installed")    print("2. Try reinstalling onnxruntime:")    print("   pip uninstall onnxruntime")    print("   pip install onnxruntime")    raise

In [ ]:
import pickleimport numpy as npimport osimport globimport cv2from tqdm import tqdmprint("=" * 60)print("Step 9: Generating Face Embeddings")print("=" * 60)face_images = glob.glob(os.path.join(FACES_DIR, "**", "*.jpg"), recursive=True)print(f"Found {len(face_images)} cropped face images")embeddings_db = {}for face_path in tqdm(face_images, desc="Generating embeddings"):    try:        img = cv2.imread(face_path)        if img is None:            continue                faces = app.get(img)        if len(faces) == 0:            continue                embedding = faces[0].embedding                rel_path = os.path.relpath(face_path, FACES_DIR)        person_name = rel_path.split(os.sep)[0]                if person_name not in embeddings_db:            embeddings_db[person_name] = []                embeddings_db[person_name].append({            'embedding': embedding,            'face_path': face_path        })        except Exception as e:        continuewith open(EMB_DB_PATH, 'wb') as f:    pickle.dump(embeddings_db, f)print("\n" + "=" * 60)print("Embeddings generation complete")print("=" * 60)print(f"Total persons: {len(embeddings_db)}")print(f"Total embeddings: {sum(len(emb) for emb in embeddings_db.values())}")print(f"Embeddings saved to: {EMB_DB_PATH}")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarityimport pickleimport numpy as npimport cv2import osprint("=" * 60)print("Step 10: Creating Face Recognition System")print("=" * 60)class FaceRecognizer:    def __init__(self, embeddings_db_path, insightface_app, threshold=0.6):        """Initialize face recognizer with embeddings database"""        with open(embeddings_db_path, 'rb') as f:            self.embeddings_db = pickle.load(f)        self.app = insightface_app        self.threshold = threshold                self.person_embeddings = {}        for person_name, embeddings_list in self.embeddings_db.items():            embeddings_array = np.array([e['embedding'] for e in embeddings_list])            avg_embedding = np.mean(embeddings_array, axis=0)            self.person_embeddings[person_name] = avg_embedding                print(f"Recognizer initialized with {len(self.person_embeddings)} known persons")        def recognize_face(self, image_path):        """Recognize a face in an image"""        img = cv2.imread(image_path)        if img is None:            return None                faces_detected = self.app.get(img)        if len(faces_detected) == 0:            return None                query_embedding = faces_detected[0].embedding                best_match = None        best_similarity = 0                for person_name, person_embedding in self.person_embeddings.items():            similarity = cosine_similarity([query_embedding], [person_embedding])[0][0]            if similarity > best_similarity:                best_similarity = similarity                best_match = person_name                if best_similarity >= self.threshold:            return {                'person_name': best_match,                'confidence': float(best_similarity),                'threshold': self.threshold            }        else:            return {                'person_name': 'Unknown',                'confidence': float(best_similarity),                'threshold': self.threshold            }        def recognize_faces_batch(self, image_paths):        """Recognize faces in multiple images"""        results = []        for img_path in image_paths:            result = self.recognize_face(img_path)            if result:                result['image_path'] = img_path            results.append(result)        return resultsrecognizer = FaceRecognizer(EMB_DB_PATH, app, threshold=0.6)print("\n" + "=" * 60)print("Face Recognition System Ready")print("=" * 60)print(f"Known persons: {len(recognizer.person_embeddings)}")print(f"Recognition threshold: {recognizer.threshold}")print("\nYou can now use the recognizer:")print("  result = recognizer.recognize_face('path/to/image.jpg')")print("  print(result)")

In [ ]:
import osimport globtest_images = glob.glob(os.path.join(DATASET_IMAGES_DIR, "**", "*.jpg"), recursive=True)[:5]if test_images and recognizer:    print("=" * 60)    print("Testing Recognition System")    print("=" * 60)        for img_path in test_images:        result = recognizer.recognize_face(img_path)        if result:            print(f"\nImage: {os.path.basename(img_path)}")            print(f"Person: {result['person_name']}")            print(f"Confidence: {result['confidence']:.4f}")            if result['person_name'] != 'Unknown':                print("Match found")            else:                print("Below threshold - Unknown person")        else:            print(f"\nImage: {os.path.basename(img_path)}")            print("No face detected")else:    print("No test images found or recognizer not initialized")

In [ ]:
import osimport globimport pickleprint("=" * 60)print("System Statistics")print("=" * 60)if os.path.exists(EMB_DB_PATH):    with open(EMB_DB_PATH, 'rb') as f:        embeddings_db = pickle.load(f)        print(f"\nDatabase Information:")    print(f"  Total persons: {len(embeddings_db)}")    print(f"  Total embeddings: {sum(len(emb) for emb in embeddings_db.values())}")        print(f"\nPer Person Statistics:")    person_stats = {}    for person_name, embeddings_list in embeddings_db.items():        person_stats[person_name] = {            'num_faces': len(embeddings_list),            'avg_confidence': 0.0        }        sorted_persons = sorted(person_stats.items(), key=lambda x: x[1]['num_faces'], reverse=True)        print(f"  Top 10 persons by face count:")    for i, (person, stats) in enumerate(sorted_persons[:10], 1):        print(f"    {i}. {person}: {stats['num_faces']} faces")        if len(sorted_persons) > 10:        print(f"    ... and {len(sorted_persons) - 10} more persons")else:    print("Embeddings database not found")if os.path.exists(LABELS_DIR):    label_files = glob.glob(os.path.join(LABELS_DIR, "*.txt"))    print(f"\nLabels Information:")    print(f"  Total label files: {len(label_files)}")        total_labels_faces = 0    for label_file in label_files:        try:            with open(label_file, 'r') as f:                lines = f.readlines()                total_labels_faces += len([l for l in lines if l.strip()])        except:            pass        print(f"  Total faces in labels: {total_labels_faces}")if os.path.exists(FACES_DIR):    face_files = glob.glob(os.path.join(FACES_DIR, "**", "*.jpg"), recursive=True)    unique_persons = [d for d in os.listdir(FACES_DIR) if os.path.isdir(os.path.join(FACES_DIR, d))]        print(f"\nCropped Faces Information:")    print(f"  Total cropped faces: {len(face_files)}")    print(f"  Unique persons: {len(unique_persons)}")        faces_per_person = {}    for face_file in face_files:        rel_path = os.path.relpath(face_file, FACES_DIR)        person_name = rel_path.split(os.sep)[0]        faces_per_person[person_name] = faces_per_person.get(person_name, 0) + 1        if faces_per_person:        avg_faces = sum(faces_per_person.values()) / len(faces_per_person)        print(f"  Average faces per person: {avg_faces:.1f}")        print(f"  Min faces per person: {min(faces_per_person.values())}")        print(f"  Max faces per person: {max(faces_per_person.values())}")print("\n" + "=" * 60)

In [ ]:
import osimport globimport pickleimport cv2import numpy as npfrom sklearn.metrics.pairwise import cosine_similarityfrom tqdm import tqdmprint("=" * 60)print("Testing Recognition Accuracy")print("=" * 60)test_images = glob.glob(os.path.join(DATASET_IMAGES_DIR, "**", "*.jpg"), recursive=True)TEST_SIZE = min(100, len(test_images))test_images = test_images[:TEST_SIZE]print(f"\nTesting on {len(test_images)} images")if 'recognizer' in globals():    print(f"Recognition threshold: {recognizer.threshold}")if not test_images or 'recognizer' not in globals():    print("No test images found or recognizer not initialized")else:    correct = 0    incorrect = 0    no_face = 0    unknown = 0        results = []        for img_path in tqdm(test_images, desc="Testing recognition"):        rel_path = os.path.relpath(img_path, DATASET_IMAGES_DIR)        path_parts = [p for p in rel_path.split(os.sep) if p]        expected_person = None                for part in reversed(path_parts[:-1]):            if not part.isdigit() and part not in ['Data base', 'Database', 'data']:                expected_person = part                break                if expected_person is None:            expected_person = path_parts[-2] if len(path_parts) >= 2 else None                result = recognizer.recognize_face(img_path)                if result is None:            no_face += 1            results.append({                'image': os.path.basename(img_path),                'expected': expected_person,                'predicted': None,                'confidence': None,                'status': 'no_face'            })        elif result['person_name'] == 'Unknown':            unknown += 1            results.append({                'image': os.path.basename(img_path),                'expected': expected_person,                'predicted': 'Unknown',                'confidence': result['confidence'],                'status': 'unknown'            })        else:            predicted_person = result['person_name']            confidence = result['confidence']                        invalid_chars = ['<', '>', ':', '"', '|', '?', '*']            if expected_person:                for char in invalid_chars:                    expected_person = expected_person.replace(char, '_')                    predicted_person = predicted_person.replace(char, '_')                                if expected_person == predicted_person:                    correct += 1                    status = 'correct'                else:                    incorrect += 1                    status = 'incorrect'            else:                incorrect += 1                status = 'no_expected'                        results.append({                'image': os.path.basename(img_path),                'expected': expected_person,                'predicted': predicted_person,                'confidence': confidence,                'status': status            })        total_with_face = correct + incorrect + unknown    if total_with_face > 0:        accuracy = (correct / total_with_face) * 100    else:        accuracy = 0        print("\n" + "=" * 60)    print("Recognition Results")    print("=" * 60)    print(f"Total images tested: {len(test_images)}")    print(f"  Correct: {correct}")    print(f"  Incorrect: {incorrect}")    print(f"  Unknown (below threshold): {unknown}")    print(f"  No face detected: {no_face}")    print(f"\nAccuracy: {accuracy:.2f}%")    print(f"  (Correct / Total with face: {correct}/{total_with_face})")        print("\n" + "=" * 60)    print("Sample Results")    print("=" * 60)        correct_examples = [r for r in results if r['status'] == 'correct'][:5]    if correct_examples:        print("\nCorrect Recognitions:")        for r in correct_examples:            print(f"  {r['image']}")            print(f"    Expected: {r['expected']}")            print(f"    Predicted: {r['predicted']} (Confidence: {r['confidence']:.4f})")        incorrect_examples = [r for r in results if r['status'] == 'incorrect'][:5]    if incorrect_examples:        print("\nIncorrect Recognitions:")        for r in incorrect_examples:            print(f"  {r['image']}")            print(f"    Expected: {r['expected']}")            print(f"    Predicted: {r['predicted']} (Confidence: {r['confidence']:.4f})")        unknown_examples = [r for r in results if r['status'] == 'unknown'][:5]    if unknown_examples:        print("\nUnknown (Below Threshold):")        for r in unknown_examples:            print(f"  {r['image']}")            print(f"    Expected: {r['expected']}")            print(f"    Confidence: {r['confidence']:.4f} (Threshold: {recognizer.threshold})")

In [ ]:
import osimport pickleimport cv2import numpy as npfrom sklearn.metrics.pairwise import cosine_similarityprint("=" * 60)print("Step 14: Adjust Recognition Threshold")print("=" * 60)if 'recognizer' in globals():    print(f"Current threshold: {recognizer.threshold}")else:    print("Recognizer not found. Please run Step 10 first.")    print("Initializing recognizer...")        if 'app' not in globals():        print("InsightFace app not found. Please run Step 8 first.")    else:        recognizer = FaceRecognizer(EMB_DB_PATH, app, threshold=0.6)        print(f"Recognizer initialized with threshold: {recognizer.threshold}")NEW_THRESHOLD = 0.6if 'app' in globals() and app is not None:    recognizer = FaceRecognizer(EMB_DB_PATH, app, threshold=NEW_THRESHOLD)    print(f"\nRecognizer updated with new threshold: {NEW_THRESHOLD}")    print("\nThreshold guidelines:")    print("  - Lower threshold (0.4-0.5): More permissive, may have false positives")    print("  - Medium threshold (0.6-0.7): Balanced, recommended")    print("  - Higher threshold (0.8-0.9): More strict, may have false negatives")else:    print("\nWarning: Cannot adjust threshold. InsightFace app not initialized.")    print("Please run Step 8 first to initialize InsightFace.")